In [ ]:
# selenium 불러오기
from selenium import webdriver
import time

# chrome창(웹드라이버) 열기
driver = webdriver.Chrome("./chromedriver")

# 실행할 웹페이지 불러오기(구글 지도 예시)
driver.get("https://www.google.com/maps/")

# 선택자로 페이지 요소 선택하기(단일)
driver.find_element_by_css_selector("선택자")

# 선택자로 페이지 요소 선택하기(다중)
driver.find_elements_by_css_selector("선택자")

# 해당 요소에 문자 입력하기(ex. 버튼)
button.send_keys("텍스트")

# 해당 요소 클릭하기
button.click()

# 의도적으로 3초 쉬어주기
time.sleep(3)

# 여러 페이지(999)에서 반복하기
for i in range(999):
    # 시간 지연
    time.sleep(3)

    # 컨테이너(가게) 데이터 수집 // div.section-result-content
    stores = driver.find_elements_by_css_selector("div.section-result-content")

    for s in stores:
        # 가게 이름 데이터 수집 // h3.section-result-title
        title = s.find_element_by_css_selector("h3.section-result-title").text

        # 평점 데이터 수집 // span.cards-rating-score
        # 평점이 없는 경우 에러 처리
        try:
            score = s.find_element_by_css_selector("span.cards-rating-score").text
        except:
            score = "평점없음"

        # 가게 주소 데이터 수집 // span.section-result-location
        addr = s.find_element_by_css_selector("span.section-result-location").text

        print(title, "/", score, "/", addr)

    # 다음페이지 버튼 클릭 하기
    # 다음페이지가 없는 경우(데이터 수집 완료) 에러 처리
    try:
        nextpage = driver.find_element_by_css_selector("button#n7lv7yjyC35__section-pagination-button-next")
        nextpage.click()
    except:
        print("데이터 수집 완료.")
        break

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
import time
import pymysql

query = '미용실' #검색어
sortingOrder = 'reviewCount' #정렬방식
arrLat = [] #위도 y
arrLng = [] #경도 x
lat = '35.179949125560846'
lng = '129.07495481148936'
location = "부산광역시청"

link = 'https://m.place.naver.com/hairshop/list?sm=mtp_hty.top&where=m&query='
link += query + '&x=' + lng + '&y=' + lat + '&level=top&entry=pll&sortingOrder=' + sortingOrder
link += '&gender=f&listViewQuery=' + query
        
# 크롬창(웹드라이버) 열기
driver = webdriver.Chrome("./chromedriver")

# 네이버 플레이스 링크 접속
driver.get(link)
time.sleep(3)

last_height = driver.find_element_by_css_selector("#_list_scroll_container > div > div > div:nth-child(2)").size
hover = driver.find_element_by_css_selector("#_list_scroll_container > div > div > div:nth-child(1) > div._15UxR > div > div > a._2QhH-._1-QNb")
hover.send_keys(Keys.ENTER)

while True:
    for i in range(30):
        hover.send_keys(Keys.PAGE_DOWN)
    time.sleep(2)
    size = driver.find_element_by_css_selector("#_list_scroll_container > div > div > div:nth-child(2)").size
    if size['height'] == last_height['height']:
        time.sleep(20)
        if size['height'] == last_height['height']:
            print("데이터 로드 완료")
            break
        print('네트워크 쫌 느림 한번 더 실행')
    last_height = size
#     print(size['height'])

stores = driver.find_elements_by_css_selector("#_list_scroll_container > div > div > div:nth-child(2) > ul > li > div._1WDkQ > div._1PT3q")
arrTitle = []
arrHref = []
for s in stores:
    title = s.find_element_by_css_selector("span._3t53y").text
    href = s.find_element_by_css_selector("a._1ptue").get_attribute('href')
    href = href.partition('hairshop/')[2].split('?')[0]
    arrTitle.append(title)
    arrHref.append(href)
# print(arrTitle)
# print(arrHref)

conn = pymysql.connect(
    charset='utf8',
    user='root', 
    passwd='', 
    host='localhost',
    db='test'
)

cursor = conn.cursor()

# # 테이블생성
# sql = '''CREATE TABLE hairshop ( 
# c_idx int(11) NOT NULL AUTO_INCREMENT PRIMARY KEY, 
# c_idx_naver int(20),
# c_name varchar(255),
# c_search_location varchar(50)
# ) 
# ''' 
# cursor.execute(sql)

# SELECT
# sql = "SELECT * FROM `hair_commerce_tb`;"
# cursor.execute(sql)
# result = cursor.fetchall()
# print(result)

for title,href in zip(arrTitle, arrHref):
    sql = "INSERT INTO hairshop (c_name, c_idx_naver, c_search_location) VALUES (%s,%s,%s)"
    val = (title,href,location)
    cursor.execute(sql, val)

conn.commit()
conn.close()
print("데이터 저장 완료")
    
# 크롬창 닫기
#driver.close()

In [ ]:
import requests
import time
import pymysql
import re
import datetime
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from fake_useragent import UserAgent

ua = UserAgent()
# ua = UserAgent(verify_ssl=False)
headers = {'User-Agent' : ua.random}

conn = pymysql.connect(
    charset='utf8',
    user='root', 
    passwd='', 
    host='localhost',
    db='test'
)
cursor = conn.cursor()

sql = "SELECT * FROM `hairshop` WHERE `c_search_location` = '부산광역시청' AND `c_idx` > 974;"
# sql = "SELECT * FROM `hairshop` WHERE `c_search_location` = '창원시청' AND `c_idx` = 670;"
cursor.execute(sql)
result = cursor.fetchall()
# print(result[0][0]) #c_idx
# print(result[0][1]) #c_idx_naver
# print(result[0][2]) #c_name
# print(result[0][3]) #c_search_location
# print(len(result))

for res in result:
    c_home = []
    c_price = []
    c_designer = []
    isDesigner = 0
    isPrice = 0

    #----------------------------home---------------------------
    url = 'https://m.place.naver.com/hairshop/{0}/home?entry=pll'.format(res[1])
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        html = response.content.decode('utf-8','replace')
        soup = BeautifulSoup(html, 'html.parser')
        try:
            c_phone = soup.select_one('#app-root > div > div.place_detail_wrapper > div.place_section.no_margin.GCwOh > div > div > div.afDDM > div > span:nth-child(1) > a').attrs['href'].split(':')[1]
        except:
            print("{0}의 c_phone 못찾음".format(url))
            c_phone = -1
        try:
            c_addr = soup.select_one('li._1M_Iz._1aj6- > div > span._2yqUQ').text
        except:
            print("{0}의 c_addr 못찾음".format(url))
            c_addr = -1
        try:
            c_hours = soup.select_one('li._1M_Iz._2KHqk > div > div > div').text
        except:
            try:
                c_hours = soup.select_one('li._1M_Iz._2KHqk > div > a > div:nth-child(1) > div > span').text
            except:
                print("{0}의 c_hours 못찾음".format(url))
                c_hours = -1
        try:
            c_score = soup.select_one('#app-root > div > div.place_detail_wrapper > div.place_section.no_margin.GCwOh > div > div > div._3XpyR > div > span._1Y6hi._1A8_M > em').text
        except:
            print("{0}의 c_score 못찾음".format(url))
            c_score = -1
        try:
            c_is_reservation = soup.select_one('#app-root > div > div.place_detail_wrapper > div.place_section.no_margin.GCwOh > div > div > div._3wP2G > div > span')
            if c_is_reservation == None:
                c_is_reservation = -1
            else:
                c_is_reservation = 1
        except:
            c_is_reservation = -1
        try:
            element = soup.select('#app-root > div > div.place_detail_wrapper > div.place_fixed_maintab > div > div > div > div > a')
            for ele in element:
                e = ele.select_one('span._3aXen').text
                if e == '스타일리스트':
                    isDesigner = 1
                if e == '가격':
                    isPrice = 1
        except:
            pass
                    
        c_home.append((c_phone, c_addr, c_hours, c_score, c_is_reservation))
        time.sleep(0.5)
    else : 
        print("페이지 로드 실패 : {0}".format(url))
        print(response.status_code)

    # ----------------------------price---------------------------
    if isPrice == 1:
        url = 'https://m.place.naver.com/hairshop/{0}/price?entry=pll'.format(res[1])
        response = requests.get(url, headers=headers)

        if response.status_code == 200:
            html = response.content.decode('utf-8','replace')
            soup = BeautifulSoup(html, 'html.parser')

            try:
                element = soup.select('#app-root > div > div.place_detail_wrapper > div:nth-child(4) > div.place_section.no_margin > div > div > ul > li')
                for ele in element:
                    try:
                        kind = ele.select_one('span._1kuzz').string
                    except:
                        print("{0}의 kind 못찾음".format(url))
                        kind = -1
                    ele_li = ele.select('ul._12NKi > li')
                    for li in ele_li:
                        try:
                            name = li.select_one('div > div > span > span').string
                        except:
                            print("{0}의 name 못찾음".format(url))
                            name = -1
                        try:
                            price = re.sub(",","",li.select_one('em._12eac').text[0:-1])
                        except:
                            print("{0}의 price 못찾음".format(url))
                            price = -1
                        c_price.append((res[1], kind, name, price))
            except:
                print("price의 element or ele_li 못 찾음 - {0}".format(url))
            time.sleep(0.5)
        else : 
            print("페이지 로드 실패 : {0}".format(url))
            print(response.status_code)

    #----------------------------stylist---------------------------
    if isDesigner == 1:
        
        url = 'https://m.place.naver.com/hairshop/{0}/stylist'.format(res[1])
        driver = webdriver.Chrome("./chromedriver")
        driver.get(url)
        time.sleep(3)

        try:
            more = driver.find_element_by_css_selector("#app-root > div > div.place_detail_wrapper > div:nth-child(5) > div > div.place_section._3a4xA > div._2kAri > a")
            more.click()
            time.sleep(0.5)
        except:
            pass

        designers = driver.find_elements_by_css_selector('#app-root > div > div.place_detail_wrapper > div:nth-child(5) > div > div.place_section._3a4xA > div:nth-child(2) > ul > li')
        for designer in designers:
            try:
                d_idx_naver = designer.find_element_by_css_selector('div._2TEwD > div.NacXe > a').get_attribute('href').partition('stylist/')[2]
            except:
                print("{0}의 d_idx_naver 못찾음".format(url))
                d_idx_naver = -1
            try:
                d_name = designer.find_element_by_css_selector('div._2TEwD > div.NacXe > a > span').text
            except:
                print("{0}의 d_name 못찾음".format(url))
                d_name = -1
            try:
                d_comment = designer.find_element_by_css_selector('div._2TEwD > div._2KAFb > a').text
            except:
                print("{0}의 d_comment 못찾음".format(url))
                d_comment = -1
            try:
                d_score = designer.find_element_by_css_selector('div._2TEwD > div._1Ezoe > a > span._3XrWs > em').text
            except:
                print("{0}의 d_score 못찾음".format(url))
                d_score = -1
            try:
                d_reservation_link = designer.find_element_by_css_selector('div.OO9oN > a').get_attribute('href')
            except:
                print("{0}의 d_reservation_link 못찾음".format(url))
                d_reservation_link = -1
            try:
                d_img_src = designer.find_element_by_css_selector('div.Deo8K > a > img').get_attribute('src')
            except:
                print("{0}의 d_img_src 못찾음".format(url))
                d_img_src = -1
            c_designer.append((d_idx_naver, res[1], d_name, d_score, d_reservation_link, d_img_src, d_comment))

        driver.close()


    # ----------------------------db저장---------------------------
#     print(c_home)
#     print(c_price)
#     print(c_designer)
#     print("'{0}'매장 데이터 저장완료".format(res[2]))
#     print("--------------------------------------------------------------------------------------")
    
    try:
        sql = "UPDATE hairshop SET c_phone=%s, c_addr=%s, c_hours=%s, c_score=%s, edit_date=%s, c_is_reservation=%s WHERE c_idx=%s"
        val = (c_home[0][0], c_home[0][1], c_home[0][2], c_home[0][3], datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S'), c_home[0][4], res[0])
        cursor.execute(sql, val)
    except:
        print("sql-Update Hairshop fail : {0}".format(val))
    
    if isPrice == 1:
        try:
            for (c_idx_naver, kind, name, price) in c_price:
                sql = "INSERT INTO price (c_idx_naver, p_kind, p_name, p_price) VALUES (%s,%s,%s,%s)"
                val = (c_idx_naver, kind, name, price)
                cursor.execute(sql, val)
        except:
            print("sql-Insert Price fail : {0}".format(val))
    
    if isDesigner == 1:
        try:
            for (d_idx_naver, c_idx_naver, d_name, d_score, d_reservation_link, d_img_src, d_comment) in c_designer:
                sql = "INSERT INTO designer (d_idx_naver, c_idx_naver, d_name, d_score, d_reservation_link, d_img_src, d_comment) VALUES (%s,%s,%s,%s,%s,%s,%s)"
                val = (d_idx_naver, c_idx_naver, d_name, d_score, d_reservation_link, d_img_src, d_comment)
                cursor.execute(sql, val)
        except:
            print("sql-Insert designer fail : {0}".format(val))
    
    conn.commit()
conn.close()

https://m.place.naver.com/hairshop/1648287197/stylist의 d_score 못찾음
https://m.place.naver.com/hairshop/1648287197/stylist의 d_score 못찾음
https://m.place.naver.com/hairshop/1648287197/stylist의 d_score 못찾음
https://m.place.naver.com/hairshop/1558594016/stylist의 d_score 못찾음
https://m.place.naver.com/hairshop/1558594016/stylist의 d_score 못찾음
https://m.place.naver.com/hairshop/1558594016/stylist의 d_score 못찾음
https://m.place.naver.com/hairshop/1558594016/stylist의 d_score 못찾음
https://m.place.naver.com/hairshop/12078226/stylist의 d_score 못찾음
https://m.place.naver.com/hairshop/12078226/stylist의 d_reservation_link 못찾음
https://m.place.naver.com/hairshop/1359022039/stylist의 d_score 못찾음
https://m.place.naver.com/hairshop/31583608/stylist의 d_score 못찾음
https://m.place.naver.com/hairshop/1599494679/stylist의 d_score 못찾음
https://m.place.naver.com/hairshop/1599494679/stylist의 d_score 못찾음
https://m.place.naver.com/hairshop/1421207239/stylist의 d_score 못찾음
https://m.place.naver.com/hairshop/33605835/stylist의 d_sc